In [1]:
import requests
import pandas as pd
from statistics import median
from parser import *

In [2]:
df = pd.read_csv("AdIndex main news 2023-12-14 - 7 days.csv")

In [7]:
test = df.loc[ : 0]

In [8]:
test

,Unnamed: 0,date,author,visitors,post_tag,post_id,link,views
0,0,2023-12-13,Данилова Светлана Викторовна,1940,Тенденции,318261,https://adindex.ru/news/tendencies/2023/12/13/...,2748


In [10]:
for link in test.link:
    print(link)
    datas = get_data_from_url(link)

https://adindex.ru/news/tendencies/2023/12/13/318261.phtml


In [11]:
datas

[{'Поисковые системы': {'Посетители': 5.0,
   'Читатели': 30.0,
   'Уникальные пользователи': 5.0,
   '% недочитываемости': 0.0,
   'Глубина просмотра': 2.0,
   'Ср. время на сайте (сек)': 16.5}},
 {'Telegram': {}}]

---

In [4]:
df = df.loc[ : 5]

In [5]:
try:
    for link in df.link:
        print(get_data_from_url(link), '14daysAgo')

except Exception as e:
    print(e)

[{'Поисковые системы': {'Посетители': 4.0, 'Читатели': 11.0, 'Уникальные пользователи': 4.0, '% недочитываемости': 0.0, 'Глубина просмотра': 2.0, 'Ср. время на сайте (сек)': 16.5}}, {'Telegram': {}}] 14daysAgo
[{'Поисковые системы': {'Посетители': 11.0, 'Читатели': 32.0, 'Уникальные пользователи': 11.0, '% недочитываемости': 0.0, 'Глубина просмотра': 3.0, 'Ср. время на сайте (сек)': 56.333333335}}, {'Telegram': {}}] 14daysAgo
[{'Поисковые системы': {'Посетители': 15.0, 'Читатели': 44.0, 'Уникальные пользователи': 15.0, '% недочитываемости': 0.0, 'Глубина просмотра': 1.5, 'Ср. время на сайте (сек)': 22.0}}, {'Telegram': {'Посетители': 2.0, 'Читатели': 2.0, 'Уникальные пользователи': 2.0, '% недочитываемости': 50.0, 'Глубина просмотра': 1.0, 'Ср. время на сайте (сек)': 7.5}}] 14daysAgo
({'query': {'ids': [22386646], 'dimensions': ['ym:s:refererDomain'], 'metrics': ['ym:s:visits', 'ym:s:pageviews', 'ym:s:users', 'ym:s:bounceRate', 'ym:s:pageDepth', 'ym:s:avgVisitDurationSeconds'], 'sort':

---

https://adindex.ru/news/hr/2023/12/13/318149.phtml
https://adindex.ru/publication/opinion/frames/2023/12/13/318148.phtml   

 При переходе на некоторые ссылки адрес меняется и мы не можем получать метрики для новости - надо менять парсер


In [22]:
get_data_from_url('https://adindex.ru/publication/opinion/frames/2023/12/13/318148.phtml', '15daysAgo')

[{'Поисковые системы': {'Посетители': 22.0,
   'Читатели': 98.0,
   'Уникальные пользователи': 21.0,
   '% недочитываемости': 0.0,
   'Глубина просмотра': 3.0,
   'Ср. время на сайте (сек)': 128.5}},
 {'Telegram': {'Посетители': 7.0,
   'Читатели': 8.0,
   'Уникальные пользователи': 7.0,
   '% недочитываемости': 0.0,
   'Глубина просмотра': 1.14,
   'Ср. время на сайте (сек)': 28.86}}]

In [109]:
page_metrics = []
response_data = response['data']
names = ['Посетители', "Читатели", "Уникальные пользователи", "% недочитываемости", "Глубина просмотра", "Ср. время на сайте (сек)"]


for data in response_data:
    page_metrics.append(
        {
            'Сайт' : data['dimensions'][0]['name']
        }
    )

    for counter, metric_name in enumerate(names):
        page_metrics[-1].update(
            {
                metric_name : data['metrics'][counter]
            }
        )

In [110]:
datas = pd.DataFrame(page_metrics)

In [111]:
datas[datas.columns[ 1 :]] = datas[datas.columns[ 1 :]].apply(lambda x: round(x, 2))

In [112]:
out = [{}, {}, {'% недочитываемости' : [], 'Глубина просмотра' : [], "Ср. время на сайте (сек)"  : []}]

restrict_keys = ['Сайт', '% недочитываемости', 'Глубина просмотра', "Ср. время на сайте (сек)"]


for data in page_metrics:
    if 'telegram' not in data['Сайт']:
        if out[0]:
            out[0].update(
                {
                    key : out[0][key] + data[key] for key in list(data.keys())[ : -3][1 :]
                } 
            )
        
        else:
            for key in data:
                if key != 'Сайт':
                    out[0][key] = data[key]


        for key in restrict_keys[1 :]:
            out[2][key].append(data[key])

    else:
        for key in data:
            if key != 'Сайт':
                out[1][key] = round(data[key], 2)

In [113]:
for key in out[2]:
    out[0][key] = median(out[2][key])

del out[2]

In [114]:
out[0] = {
    'Поисковые системы' : out[0]
}

out[1] = {
    'Telegram' : out[1]
}

In [115]:
out

[{'Поисковые системы': {'Посетители': 466.0,
   'Читатели': 669.0,
   'Уникальные пользователи': 446.0,
   '% недочитываемости': 0.0,
   'Глубина просмотра': 1.0,
   'Ср. время на сайте (сек)': 15.0}},
 {'Telegram': {'Посетители': 3.0,
   'Читатели': 3.0,
   'Уникальные пользователи': 3.0,
   '% недочитываемости': 33.33,
   'Глубина просмотра': 1.0,
   'Ср. время на сайте (сек)': 10.33}}]

In [118]:
pd.DataFrame(out)

,Поисковые системы,Telegram
0,"{'Посетители': 466.0, 'Читатели': 669.0, 'Уник...",NaN
1,NaN,"{'Посетители': 3.0, 'Читатели': 3.0, 'Уникальн..."
